# ICOER Text Analysis Notebook
Análise de frases com vetorização POS, cálculo da métrica ICOER e exportação CSV.

In [ ]:
!pip install -q spacy pandas matplotlib
!python -m spacy download pt_core_news_sm

In [ ]:
import spacy
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter

In [ ]:
# Carregar modelo SpaCy
nlp = spacy.load("pt_core_news_sm")

In [ ]:
# Verificar entrada via widget ou usar lista estática de frases
try:
    if isinstance(frases, list) and all(isinstance(f, str) for f in frases):
        pass  # já está certo
    else:
        raise ValueError
except:
    frases = [f.strip() for f in text_input.value.split('.') if f.strip()]
frases


In [ ]:
# Análise de tags
data = []
for frase in frases:
    doc = nlp(frase)
    tags = [token.pos_ for token in doc]
    tag_counts = dict(Counter(tags))
    data.append({
        "frase": frase,
        "n_palavras": len(doc),
        "tags": tag_counts
    })
df = pd.DataFrame(data)
df

In [ ]:
# Vetorização POS para cálculo do ICOER (simples: frequência POS normalizada)
def vetor_pos(tags_dict):
    base = ['NOUN', 'VERB', 'ADJ', 'ADV', 'ADP', 'PROPN']
    total = sum(tags_dict.values())
    return [tags_dict.get(pos, 0)/total if total > 0 else 0 for pos in base]

df['vetor'] = df['tags'].apply(vetor_pos)

In [ ]:
# Cálculo do ICOER como 1 - entropia POS (quanto mais uniforme, menor a coerência)
import numpy as np

def entropia(vetor):
    vetor = np.array(vetor)
    vetor = vetor[vetor > 0]
    return -np.sum(vetor * np.log2(vetor)) if len(vetor) > 0 else 0

df['icoer'] = df['vetor'].apply(lambda v: round(1 - entropia(v), 4))
df[['frase', 'icoer']]

In [ ]:
# Exportação CSV
df[['frase', 'n_palavras', 'tags', 'icoer']].to_csv("icoer_text_analysis.csv", index=False)
df